# Qwen 0.5b on GRPO 모델 훈련

---
이 노트북은 [Original Notebook](https://colab.research.google.com/drive/1bfhs1FMLW3FGa8ydvkOZyBNxLYOu0Hev?usp=sharing#scrollTo=Q7qTZbUcg5VD) 인 gsm8k 수학 데이터셋을 사용하여 Qwen-0.5b 를 학습시키는 내용을 SageMaker 에서 재구성한 내용 입니다.
실제로 [QRPO](https://medium.com/data-science-in-your-pocket/what-is-grpo-the-rl-algorithm-used-to-train-deepseek-12acc19798d3) 를 사용한 학습을 SageMaker Training Job 을 실행 하는 노트북 입니다.

### 노트북 실험 환경
- SageMaker Studio 의 [Code Editor](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/code-editor.html) 의 ml.g5.12xlarge 를 사용하였습니다.

---

## 1. 환경 설정
- 시작을 위해서 여기 가이드 보세요: [Setup Guide](../setup/README.md)
- 이후에 아래 셀 실행을 통해서, 필요한 패키지가 설치 되었는지를 확인 합니다.

In [6]:
! pip list | grep -E "sagemaker|vllm|trl|datasets"

datasets                           3.2.0
sagemaker                          2.232.2
sagemaker-core                     1.0.21
sagemaker-mlflow                   0.1.0
trl                                0.14.0
vllm                               0.7.2


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 훈련 작업 생성하기

실제 [SageMaker CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html)는 SageMaker Python SDK의 [상위 레벨 'Estimator' 클래스](https://sagemaker.readthedocs.io/en/stable/overview.html)가 단순화하는 데 도움이 되는 여러 저수준 세부 사항을 필요로 합니다. 특히:
- 정확한 컨테이너 이미지 URI를 지정하는 대신, SDK가 선택된 프레임워크와 버전을 기반으로 이를 찾아줍니다
- SDK는 우리의 `scripts` 번들을 자동으로 압축하여 S3에 업로드하고, 훈련 작업이 거기서 로드하도록 구성합니다

따라서 먼저, 작업을 구성하고 어떤 인프라(컴퓨팅 인스턴스의 수와 유형)에서 실행되어야 하는지를 지정하는 `estimator` 객체를 생성합니다:

> ℹ️ 다른 작업을 대신 실행하는 서비스들처럼, 훈련 작업은 S3의 입력 훈련 데이터와 같은 리소스에 접근할 수 있도록 [IAM 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)을 가정합니다. SageMaker 노트북 자체가 이미 가정된 역할로 실행되고 있으므로, 단순화를 위해 훈련 작업 역할을 노트북 역할과 동일하게 설정할 것입니다.

- 이 노트북에서는 훈련 데이터 셋을 S3 에 업로드 하지 않고, 간소화 목적으로 train.py 에서 직접 데이타를 다운로드 해서 훈련하는 형태로 사용합니다.


### train.sh 파일 구조

아래 Estimator 에서 entry_point 로 train.sh 를 사용하였습니다. 내용은 아래와 같습니다. 
- 구조는 훈련에 필요한 Python Package 설치 와 train.py 를 실행하는 것으로 구성 되어 있습니다.
- 일반적으로 Python Package 설치는 requirements.txt 에서 사용가능하나, 특정 Python Repo 에서 다운로드를 지정하는 --index-url https://download.pytorch.org/ 를 사용할 수 없습니다. 그래서 여기서는 train.sh 에서 이 역할을 수행합니다.
- num_generations 4 의 인자는 vllm 에서 response 를 몇 개 생성을 하는지를 나타내는 파라라미터 입니다. 아래는 특정 인스턴스에서 실험한 결과 입니다.
    - num_generations=16, # success on ml.p4de.24xlarge
    - num_generations=4,  # success on ml.p4d.24xlarge
    - num_generations=6,  # failure on ml.p4d.24xlarge
    - num_generations=8,  # failuer on ml.p4d.24xlarge
    - num_generations=2,  # success on ml.g5.12xlarge

    ```train.sh
    #!/bin/bash
    pip install torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
    pip install transformers==4.48.3 vllm==0.7.2 trl==0.14.0 datasets==3.2.0
    python train.py num_generations 4
    ```

In [9]:
import sagemaker
from sagemaker.pytorch import PyTorch

nb_role = sagemaker.get_execution_role()

#instance_type="ml.p4d.24xlarge"
instance_type="ml.p4de.24xlarge"

estimator = PyTorch(
    source_dir="scripts",
    entry_point="train.sh",
    framework_version="2.0.1",
    py_version="py310",
    base_job_name="qrpo-qwen",
    instance_count=1,
    instance_type=instance_type,
    keep_alive_period_in_seconds=3600,
    role=nb_role,
)

구성이 완료되면, 입력 데이터 위치를 지정하고 `estimator.fit()`을 실행하여 실제 훈련 작업을 시작할 수 있습니다.

데이터 입력 "채널"의 수, 이름 및 유형은 [사용자가 결정](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html#sagemaker-CreateTrainingJob-request-InputDataConfig)할 수 있습니다: 단지 노트북이 스크립트에서 기대하는 것과 동일한 채널을 구성하도록 하면 됩니다.

In [10]:
%%time

estimator.fit(
    wait=True,  # Wait for the training to complete (default=True)
    logs=True,  # Stream training job logs to the notebook (default=True, requires wait=True)
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: qrpo-qwen-2025-03-03-10-11-07-368


2025-03-03 10:11:12 Starting - Starting the training job
2025-03-03 10:11:12 Pending - Training job waiting for capacity..................
2025-03-03 10:13:47 Pending - Preparing the instances for training.....................
2025-03-03 10:17:32 Downloading - Downloading the training image..................
2025-03-03 10:20:19 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-03-03 10:21:15,882 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-03-03 10:21:16,001 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-03 10:21:16,010 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-03-03 10:21:16,012 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-03-03 10:21:17,488 sagemaker-training-to